In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime


In [4]:


tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']
end_date = datetime.now().strftime('%Y-%m-%d')

start_date = (datetime.now() - pd.DateOffset(years=2)).strftime('%Y-%m-%d') 
raw_data = yf.download(tickers, start=start_date, end=end_date)

df = raw_data.stack(level=1).reset_index()
df = df.rename(columns={'level_0': 'Date', 'level_1': 'Ticker'})
print(df.head())


C:\Users\bilal\AppData\Local\Temp\ipykernel_7772\544362269.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  4 of 4 completed

Price       Date Ticker       Close        High         Low        Open  \
0     2023-11-07   AAPL  180.056381  180.670362  177.234020  177.441974   
1     2023-11-07   AMZN  142.710007  143.369995  140.500000  140.550003   
2     2023-11-07  GOOGL  130.072876  131.006440  128.990346  129.814662   
3     2023-11-07   MSFT  355.177521  357.078860  352.320581  354.064292   
4     2023-11-08   AAPL  181.115967  181.670532  179.828574  180.581211   

Price    Volume  
0      70530000  
1      53553500  
2      29757300  
3      25833900  
4      49340300  



C:\Users\bilal\AppData\Local\Temp\ipykernel_7772\544362269.py:7: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = raw_data.stack(level=1).reset_index()


In [5]:
kpi_list = []

for ticker in tickers:
    ticker_data = df[df['Ticker'] == ticker].copy()
    
    
    latest_price = ticker_data.iloc[-1]['Close']
    kpi_list.append({'Ticker': ticker, 'KPI_Name': 'Latest Price', 'KPI_Value': latest_price})

    
    current_year = datetime.now().year
    ytd_data = ticker_data[ticker_data['Date'].dt.year == current_year]
    if not ytd_data.empty and len(ytd_data) > 1:
        start_price = ytd_data.iloc[0]['Close']
        current_price = ytd_data.iloc[-1]['Close']
        ytd_return = (current_price - start_price) / start_price * 100
        kpi_list.append({'Ticker': ticker, 'KPI_Name': 'YTD Return (%)', 'KPI_Value': ytd_return})

    
    rolling_52_week = ticker_data.tail(252)
    week_52_high = rolling_52_week['High'].max()
    week_52_low = rolling_52_week['Low'].min()
    kpi_list.append({'Ticker': ticker, 'KPI_Name': '52-Week High', 'KPI_Value': week_52_high})
    kpi_list.append({'Ticker': ticker, 'KPI_Name': '52-Week Low', 'KPI_Value': week_52_low})

    
    if len(ticker_data) >= 5:
        price_5_days_ago = ticker_data.iloc[-5]['Close']
        price_change_5d = (latest_price - price_5_days_ago) / price_5_days_ago * 100
        kpi_list.append({'Ticker': ticker, 'KPI_Name': '5-Day Change (%)', 'KPI_Value': price_change_5d})

    #
    avg_volume_30d = ticker_data.tail(30)['Volume'].mean()
    kpi_list.append({'Ticker': ticker, 'KPI_Name': 'Avg Volume (30d)', 'KPI_Value': avg_volume_30d})

kpi_df = pd.DataFrame(kpi_list)

kpi_df.to_csv('stock_kpis.csv', index=False)